In [1]:
# Real-Time Sign Language Object Detection GUI Version

import cv2
from ultralytics import YOLO
import threading
from tkinter import *
from PIL import Image, ImageTk

# Load your trained YOLOv8 model
model = YOLO('best.pt')
class_names = ['Dance', 'Fourteen', 'Mustache', 'Sweetheart', 'Turtle']

# GUI app class
class SignLanguageDetectorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Sign Language Detection")
        self.root.geometry("800x600")

        self.video_label = Label(root)
        self.video_label.pack()

        self.btn_frame = Frame(root)
        self.btn_frame.pack(pady=10)

        self.start_btn = Button(self.btn_frame, text="Start Detection", command=self.start_detection)
        self.start_btn.pack(side=LEFT, padx=10)

        self.stop_btn = Button(self.btn_frame, text="Stop", command=self.stop_detection, state=DISABLED)
        self.stop_btn.pack(side=LEFT, padx=10)

        self.cap = None
        self.running = False

    def start_detection(self):
        self.running = True
        self.cap = cv2.VideoCapture(0) # Could be 0, 1, or 2 depending on webcam
        self.start_btn.config(state=DISABLED)
        self.stop_btn.config(state=NORMAL)
        threading.Thread(target=self.detect_loop).start()

    def stop_detection(self):
        self.running = False
        if self.cap:
            self.cap.release()
        self.start_btn.config(state=NORMAL)
        self.stop_btn.config(state=DISABLED)

    def detect_loop(self):
        while self.running:
            ret, frame = self.cap.read()
            if not ret:
                break

            results = model.predict(source=frame, conf=0.4, verbose=False)
            boxes = results[0].boxes

            for box in boxes:
                x1, y1, x2, y2 = map(int, box.xyxy[0])
                conf = float(box.conf[0])
                cls = int(box.cls[0])

                label = f"{class_names[cls]} {conf:.2f}"
                cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(frame, label, (x1, y1 - 10),
                            cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

            # Convert to RGB and display in Tkinter
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = Image.fromarray(rgb_frame)
            imgtk = ImageTk.PhotoImage(image=img)

            self.video_label.imgtk = imgtk
            self.video_label.configure(image=imgtk)

        self.cap.release()

    def on_close(self):
        self.stop_detection()
        self.root.destroy()

# Run the GUI
root = Tk()
app = SignLanguageDetectorApp(root)
root.protocol("WM_DELETE_WINDOW", app.on_close)
root.mainloop()

2025-03-23 14:58:53.132 python[6478:1484980] +[IMKClient subclass]: chose IMKClient_Modern
2025-03-23 14:58:53.132 python[6478:1484980] +[IMKInputSession subclass]: chose IMKInputSession_Modern
2025-03-23 14:58:56.644 python[6478:1484980] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
